In [9]:
import numpy as np
import cv2
from PIL import Image

In [12]:
path = '/home/hinton/NAS_AIlab_dataset/dataset/cifar10/train/cat/house_cat_s_002092.png'
img = cv2.imread(path)
img2 = Image.open(path)
print(img.shape)
print(img2)

(32, 32, 3)
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7F87FA2ED290>
